In [174]:
from math import floor

class field(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
        self.generated_terms = []
        self.connected = [] #list of connected terms, includes repeats
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) #+ '\n' \
            #+ 'connected: ' + str(self.connected)
    def __eq__(self, other):
        eq = (self.elementary_str == other.elementary_str) & (self.contractions == other.contractions)
        return eq
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    def get_connected(self):
        return self.connected
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field1 = field(['P'],[], 'P')
                self.elementary.append(field1)
            elif symbol == 'DP':
                field1_deriv = field(['DP'],[], 'DP')
                self.elementary.append(field1_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    def set_connected(self, connected_list):
        self.connected = connected_list
    
    
    def make_symbol(self): #use for connected terms
        symbol_str = ''
        for term in self.get_elementary_str():
            symbol_str = symbol_str + term
        return symbol_str
            
    
    def print_connected(self):
        for term in self.get_connected():
            print(term)
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        #print("counter1: " + str(counter1))
        #print("counter2: " + str(counter2))
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    def bubbleSort(self, field_list): #sorts list of connected fields in ascending order by massDim. 
        n = len(field_list)
 
        # Traverse through all array elements
        for i in range(n):
 
            # Last i elements are already in place
            for j in range(0, n-i-1):
 
                # traverse the array from 0 to n-i-1
                # Swap if the element found is greater
                # than the next element
                if field_list[j].massDim() > field_list[j+1].massDim():
                    field_list[j], field_list[j+1] = field_list[j+1], field_list[j]
    
        return field_list
    
    
    def multiply(self, other, symbol_prod=''):
        '''
        Multiplies without contraction
        INPUT:
        self: field object
        other: field object
        symbol_prod: symbol for new field obj output
        
        OUTPUT:
        prod: field object, concatenating self and other, by inserting [0,0] in contractions
        
        '''
        if (not self.get_elementary_str()) or (not other.get_elementary_str()):
            #print("one factor is const")
            contractions_prod = self.get_contractions() + other.get_contractions()
        else:
            contractions_prod = self.get_contractions() + [[0,0]] + other.get_contractions()
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        connected_prod = self.get_connected() + other.get_connected()
        #Sort connected_prod by mass dimension, with lowest first and highest last
        connected_prod_sorted = field.bubbleSort(self, connected_prod)
        
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        prod.set_connected(connected_prod_sorted)
        
        return prod
    
    
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate connected field terms
        
        OUTPUT
        terms: list of field terms of mass dimension no larger than d.  
        '''
        
        const = field([],[],'1')
        const.set_connected([field([],[],'1')])
        terms = [const]
        
        #print("in generate_connected")
        
        if d >= 2:
            #print("in d>=2")
            field1 = field(['P'],[],'P')
            field2 = field(['P'],[],'P')
            prod = field.contract(field1, field2, [0,1], 'PP')
            prod.set_connected([field.contract(field1, field2, [0,1], 'PP')])
            terms.append(prod)
        if d >= 4:
            deriv1 = field(['DP'],[],'DP')
            deriv2 = field(['DP'],[],'DP')
            prod = field.contract(deriv1, deriv2, [1,1], 'DPDP')
            prod.set_connected([field.contract(deriv1, deriv2, [1,1], 'DPDP')])
            terms.append(prod)
        
        field1 = field(['P'],[],'P')
        deriv1 = field(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = field.contract(field1, deriv1, [0,1], 'PDP')
        
        while root.massDim() <= d:
            #print("")
            #print("root: ")
            #print(root)
            #print("")
            #root_symbol = root.get_symbol()
            #print("root_symbol: " +str(root_symbol))
            #print("root_massDim: " +str(root.massDim()))
            
            if root.LORrank() != 0:
                #print("in root.LORrank()!=0")
                deriv1 = field(['DP'],[],'DP')
                root = field.contract(root, deriv1, [1,0], '')
                #print("After DP LOR contraction, root_symbol: " +str(root.get_symbol()))
                #print("")
                #print("root: ")
                #print(root)
                #print("")
            
            if root.ONrank() != 0:
                #print("in root.ONrank()!=0")
                field1 = field(['P'],[],'P')
                deriv1 = field(['DP'],[],'DP')
                new = field.contract(root, field1, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    #print("in new term added")
                    new.set_connected([field.contract(root, field1, [0,1], '')])
                    terms.append(new)
                
                root = field.contract(root, deriv1, [0,1], '')
                #print("After DP ON contraction, root_symbol: " +str(root.get_symbol()))
                #print("")
                #print("root: ")
                #print(root)
                #print("")    
        
        return terms
    
   
    
    def generate(self, D):
        if D < 2:
            print("D should be an integer >= 2.")
            return []
        else:
            #initiate list of generated terms
            generated = []
            
            #build list of connected terms
            connected_terms_D = self.generate_connected(D)
            for term in connected_terms_D:
                print("term.get_connected()")
                print(term.get_connected()[0].get_symbol())
            
            #initiate cache
            const = field([],[],'')
            const.set_connected([field([],[],'')])
            #print("const.get_connected(): " + str(const.get_connected()))
            cache = []
            for term in connected_terms_D:
                cache.append(term)
            
            for i in range(floor(D/2)-1):
                print("")
                print("i: " + str(i))
                print("")
                print("len(cache): " + str(len(cache)))
                print("len(connected_terms_D): " + str(len(connected_terms_D)))
                for j in range(len(cache)):
                    for k in range(len(connected_terms_D)):
                        print("j: " + str(j))
                        print("k: " + str(k))
                        print("")
                        if connected_terms_D[k].massDim() <= cache[j].get_connected()[0].massDim():
                            print("In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' ")
                            prod = field.multiply(cache[j], connected_terms_D[k])
                            cache.append(prod)
                        else: 
                            pass
                
             
            for term in cache:
                if term.massDim() <= D:
                    print("In 'prod.massDim() <= D' ")
                    print("term.massDim(): " + str(term.massDim()))
                    print("term.get_elementary_str(): " + str(term.get_elementary_str()))
                    generated.append(prod)
                else:
                    pass
            
                
            return generated
                    
            

In [175]:
A = field([],[],'')

In [176]:
A.set_elementary()

In [177]:
generated_list = A.generate(6)

term.get_connected()
1
term.get_connected()
PP
term.get_connected()
DPDP
term.get_connected()


i: 0

len(cache): 4
len(connected_terms_D): 4
j: 0
k: 0

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 0
k: 1

j: 0
k: 2

j: 0
k: 3

j: 1
k: 0

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 1
k: 1

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 1
k: 2

j: 1
k: 3

j: 2
k: 0

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 2
k: 1

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 2
k: 2

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 2
k: 3

j: 3
k: 0

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 3
k: 1

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 3
k: 2

In 'connected_terms_D[k].massDim() <= cache.get_connected()[0].massDim():' 
j: 3
k: 3

In [178]:
print(len(generated_list))
for term in generated_list:
    print(term)
    print('')

23
elementary_str: ['P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 18
LORrank: 0
ONrank: 0

elementary_str: ['P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 18
LORrank: 0
ONrank: 0

elementary_str: ['P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 18
LORrank: 0
ONrank: 0

elementary_str: ['P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1], [0, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 18
LORrank: 0
ONrank: 0

elementary_str: ['P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P', 'P', 'DP', 'DP', 'P'

In [158]:
connected_list = A.generate_connected(14)

In [159]:
for term in connected_list:
    print(term)
    print(term.make_symbol())
    print(len(term.get_elementary()))
    print(len(term.get_connected()))
    
    print("")

elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0

0
1

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
PP
2
1

elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: DPDP
massDim: 4
LORrank: 0
ONrank: 0
DPDP
2
1

elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
PDPDPP
4
1

elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0
PDPDPDPDPP
6
1

elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 14
LORrank: 0
ONrank: 0
PDPDPDPDPDPDPP
8
1

